<a href="https://colab.research.google.com/github/dobrotvorn/DataCon23/blob/main/Mini_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Описание задания**
# Часть 1. Работа с данными

1) Просмотреть датасет на наличие выбросов и неверных данных

2) Разобраться в графиках из бейзлайна

3) Построить свои графики и выдвинуть по ним гипотезы

4) Отталкиваясь от этого, добавить некоторое количество новых дескрипторов при помощи rdkit (главное разобраться в коде, который это делает)

# Часть 2. Построение моделей

1) Построить свои модели на основе базовых регрессий из scikit-learn (минимум: обойти бейзлайн и попробовать хотя бы 3-4 модели
отличные от него)

2) Попробовать поработать с изменением параметров моделей и изменить соотношение тестовой/тренировочной выборки

3) Вывести графики feature importance полученных моделей

# Часть 3. Tensorflow

1) Провести эксперименты с архитектурами: количество слоев; функции активации; добавление дроп-аута/нормализации



## **Загрузка данных и библиотек**
# Импортируем и скачиваем библиотеки

In [ ]:
!pip install gdown -- quiet
!pip install fancyimpute --quiet
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import plotly
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import KNNImputer
from IPython.display import display, HTML
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from fancyimpute import KNN

ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet


In [ ]:
# тут захардкодим статичные переменные
global ordinal_encoder
global text_columns
global path2firstDb
global numerics
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
path2firstDb = '/content/firstDb.csv' if 'google.colab' in sys.modules else 'firstDb.csv'

# Скачиваем данные

In [ ]:
url1 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR7whTOZIO16kGSfmtP3zHyXoRog0r6LVk_UZjpIhHwiRocAL5SIiqUyoBfpFXqe1fnyDsIOhTc7cjM/pub?gid=139006229&single=true&output=csv'
output1 = 'firstDb.csv'
gdown.download(url1, output1, quiet=True)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(


'firstDb.csv'

## **Часть 1. Работа с ДС**

In [ ]:
fdb = pd.read_csv(path2firstDb)
fdb.head(7)

,SMILES,Solubility,MolLogP,MolMR,HeavyAtomCount,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,TPSA,LabuteASA,BertzCT
0,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,10.00000,102.4454,23.0,17.0,142.0,0.0,0.00,158.520601,210.377334
1,O=C1Nc2cccc3cccc1c23,-3.254767,2.40550,51.9012,13.0,0.0,62.0,2.0,29.10,75.183563,511.229248
2,Clc1ccc(C=O)cc1,-2.177078,2.15250,36.8395,9.0,1.0,46.0,1.0,17.07,58.261134,6085.486839
3,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,8.11610,200.7106,53.0,10.0,264.0,6.0,120.72,323.755434,1964.648666
4,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,2.48540,119.0760,31.0,12.0,164.0,2.0,56.60,183.183268,769.899934
5,Cc1cccc(C=C)c1,-3.123150,2.63802,41.2700,9.0,1.0,46.0,1.0,0.00,55.836626,211.033225
6,CCC1CCC(CCC(O)=O)C1,-3.286116,2.67750,47.9918,12.0,4.0,70.0,0.0,37.30,73.973655,153.917569
